<a href="https://colab.research.google.com/github/Palaeoprot/IPA/blob/main/Skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#@title #Import Libraries and Setup
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import math
import json
import gspread
from datetime import datetime
import random
from scipy.optimize import differential_evolution, curve_fit
from scipy.interpolate import interp1d
import logging
from typing import List, Dict, Tuple
import configparser
import argparse

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
import numpy as np

class ModulAAR:
    def __init__(self, config):
        self.config = config
        self.data = None
        self.processed_data = None
        self.logger = self._setup_logger()

    def _setup_logger(self):
        from datetime import datetime
        return lambda msg, level="INFO": print(f"{datetime.now()} - {level} - {msg}")

    def load_data(self, source, is_gsheet=False):
        """Load data from a CSV file or Google Sheets."""
        try:
            if is_gsheet:
                sheet_id = source['sheet_id']
                gid = source['gid']
                export_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
                self.data = pd.read_csv(export_url)
                self.logger(f"Data loaded successfully from Google Sheet")
            else:
                self.data = pd.read_csv(source)
                self.logger(f"Data loaded successfully from {source}")
        except Exception as e:
            self.logger(f"Error loading data: {str(e)}", "ERROR")
            self.data = None
        return self.data

    def process_data(self, df=None):
        """Process the loaded data based on its type."""
        if df is None:
            df = self.data
        if df is None:
            self.logger("No data available for processing.", "ERROR")
            return None

        df = df.dropna()

        # Flexible column renaming
        rename_mapping = {
            'time (h)': 'time',
            'temp': 'temp (°C)',
            'Pre-heat bleach time': 'Pre-heat bleach time (h)'
            # Add other potential column mappings here if needed
        }
        df.rename(columns=rename_mapping, inplace=True)

        expected_eggshell_columns = [
            'Pre-heat bleach time (h)', 'temp (°C)', 'pH', 'sample', 'time',
            '[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]',
            'Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L'
        ]
        expected_betalactoglobulin_columns = ['Condition', 'Fraction', 'm']

        if all(col in df.columns for col in expected_eggshell_columns):
            self.logger("Processing eggshell data")
            return self._process_eggshell_data(df)
        elif all(col in df.columns for col in expected_betalactoglobulin_columns):
            self.logger("Processing beta-lactoglobulin data")
            return self._process_betalactoglobulin_data(df)
        else:
            self.logger(
                f"Unknown data format. Expected columns for eggshell: {expected_eggshell_columns}, "
                f"or beta-lactoglobulin: {expected_betalactoglobulin_columns}. Found columns: {list(df.columns)}",
                "ERROR"
            )
            return None

    def _process_eggshell_data(self, df):
        """Process eggshell data."""
        df = self.clean_data(df)
        grouped_faa, grouped_thaa = self.calculate_stats(df)

        processed_df = pd.merge(grouped_faa, grouped_thaa, on=['temp (°C)', 'time'], how='outer')

        # Add 'temp (K)' column
        if 'temp (°C)' in processed_df.columns:
            processed_df['temp (K)'] = processed_df['temp (°C)'] + 273.15

        self.logger("Eggshell data processed successfully")
        return processed_df

    def _process_betalactoglobulin_data(self, df):
        """Process beta-lactoglobulin data."""
        relevant_columns = ['Condition', 'Fraction', 'm', '[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]',
                            'Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L']
        df = df[relevant_columns]

        # Convert concentration and D/L columns to numeric
        for col in df.columns:
            if col.startswith('[') or col.endswith('D/L'):
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Group by condition and calculate statistics
        grouped = df.groupby(['Condition', 'Fraction', 'm']).agg({
            col: ['mean', 'std'] for col in df.columns if col not in ['Condition', 'Fraction', 'm']
        }).reset_index()

        # Flatten column names
        grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

        self.logger("Beta-lactoglobulin data processed successfully")
        return grouped

    def clean_data(self, df):
        """Clean the eggshell data by converting to numeric and filtering."""
        concentration_columns = ['[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]']
        dl_columns = ['Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L']

        for col in concentration_columns + dl_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        df = df.dropna(subset=concentration_columns + dl_columns)
        return df

    def calculate_stats(self, df):
        """Calculate statistics for FAA and THAA samples."""
        faa_df = df[df['sample'] == 'FAA']
        thaa_df = df[df['sample'] == 'THAA']

        grouped_faa = faa_df.groupby(['temp (°C)', 'time']).agg({
            '[Asx]': ['mean', 'std', 'count'],
            '[Glx]': ['mean', 'std', 'count'],
            '[Ser]': ['mean', 'std', 'count'],
            '[Ala]': ['mean', 'std', 'count'],
            '[Val]': ['mean', 'std', 'count'],
            '[Phe]': ['mean', 'std', 'count'],
            '[Ile]': ['mean', 'std', 'count'],
            'Asx D/L': ['mean', 'std', 'count'],
            'Glx D/L': ['mean', 'std', 'count'],
            'Ser D/L': ['mean', 'std', 'count'],
            'Ala D/L': ['mean', 'std', 'count'],
            'Val D/L': ['mean', 'std', 'count'],
            'Phe D/L': ['mean', 'std', 'count'],
            'Ile D/L': ['mean', 'std', 'count']
        }).reset_index()

        grouped_thaa = thaa_df.groupby(['temp (°C)', 'time']).agg({
            '[Asx]': ['mean', 'std', 'count'],
            '[Glx]': ['mean', 'std', 'count'],
            '[Ser]': ['mean', 'std', 'count'],
            '[Ala]': ['mean', 'std', 'count'],
            '[Val]': ['mean', 'std', 'count'],
            '[Phe]': ['mean', 'std', 'count'],
            '[Ile]': ['mean', 'std', 'count'],
            'Asx D/L': ['mean', 'std', 'count'],
            'Glx D/L': ['mean', 'std', 'count'],
            'Ser D/L': ['mean', 'std', 'count'],
            'Ala D/L': ['mean', 'std', 'count'],
            'Val D/L': ['mean', 'std', 'count'],
            'Phe D/L': ['mean', 'std', 'count'],
            'Ile D/L': ['mean', 'std', 'count']
        }).reset_index()

        new_columns_faa = ['temp (°C)', 'time'] + [
            f'{aa}_Conc_FAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']] + [
            f'{aa}_D/L_FAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']]
        new_columns_thaa = ['temp (°C)', 'time'] + [
            f'{aa}_Conc_THAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']] + [
            f'{aa}_D/L_THAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']]

        grouped_faa.columns = new_columns_faa
        grouped_thaa.columns = new_columns_thaa

        for col in new_columns_faa:
            if '_Std' in col:
                count_col = col.replace('_Std', '_Count')
                grouped_faa.loc[grouped_faa[count_col] == 1, col] = 0

        for col in new_columns_thaa:
            if '_Std' in col:
                count_col = col.replace('_Std', '_Count')
                grouped_thaa.loc[grouped_thaa[count_col] == 1, col] = 0

        return grouped_faa, grouped_thaa

    def calculate_amino_acid_distribution(self, df, amino_acid):
        """Calculate the distribution of a given amino acid."""
        if df is None:
            self.logger("No processed data available. Please process data first.", "ERROR")
            return None

        amino_acid_distributions = []

        initial_thaa = df[f"{amino_acid}_Conc_THAA_Mean"].iloc[0]

        for index, row in df.iterrows():
            thaa = row[f"{amino_acid}_Conc_THAA_Mean"]
            faa = row[f"{amino_acid}_Conc_FAA_Mean"]
            thaa_dl = row[f"{amino_acid}_D/L_THAA_Mean"]
            faa_dl = row[f"{amino_acid}_D/L_FAA_Mean"]
            time_point = row["time"]

            # Calculate BAA, FAA, and losses
            baa = thaa - faa
            baa_d = baa * thaa_dl / (1 + thaa_dl)
            baa_l = baa - baa_d
            faa_d = faa * faa_dl / (1 + faa_dl)
            faa_l = faa - faa_d
            total_loss = max(initial_thaa - thaa, 0)
            loss_d = total_loss * faa_dl / (1 + faa_dl) if total_loss > 0 else 0
            loss_l = total_loss - loss_d if total_loss > 0 else 0

            # Calculate standard deviations
            thaa_std = row[f"{amino_acid}_Conc_THAA_Std"]
            faa_std = row[f"{amino_acid}_Conc_FAA_Std"]
            baa_std = np.sqrt(thaa_std**2 + faa_std**2)
            baa_d_std = baa_std * thaa_dl / (1 + thaa_dl)
            baa_l_std = baa_std - baa_d_std
            faa_d_std = faa_std * faa_dl / (1 + faa_dl)
            faa_l_std = faa_std - faa_d_std
            loss_std = np.sqrt(2 * thaa_std**2)
            loss_d_std = loss_std * faa_dl / (1 + faa_dl)
            loss_l_std = loss_std - loss_d_std

            amino_acid_distributions.append({
                "time": time_point,
                "temp (K)": row["temp (K)"],
                "BAA_D": baa_d, "BAA_L": baa_l,
                "BAA_D_Std": baa_d_std, "BAA_L_Std": baa_l_std,
                "FAA_D": faa_d, "FAA_L": faa_l,
                "FAA_D_Std": faa_d_std, "FAA_L_Std": faa_l_std,
                "FAA_D_loss": loss_d, "FAA_L_loss": loss_l,
                "FAA_D_loss_Std": loss_d_std, "FAA_L_loss_Std": loss_l_std,
            })

        return pd.DataFrame(amino_acid_distributions)


# Usage
config = {
    'N': 50000,
    'fold_water': 8,
    'temperature_kelvin': 353.15,
    'amino_acid': 'Asx',
    'initial_length': 1500,
    'rate_params': {
        # Add rate parameters here
    }
}

model = ModulAAR(config)

# Load data from Google Sheets
sheet_info = {'sheet_id': "1nA6jSAkAf1Ud-kHdaYTMtBPgKhe9nBg_IjM9idLlj8E", 'gid': "1259514505"}
df1 = model.load_data(sheet_info, is_gsheet=True)

# Process data
if df1 is not None:
    processed_df1 = model.process_data(df1)
    if processed_df1 is not None:
        print(processed_df1.head())

        # Calculate amino acid distribution for Asx
        asx_distribution = model.calculate_amino_acid_distribution(processed_df1, 'Asx')
        print(asx_distribution.head())


2024-07-05 18:22:05.963365 - INFO - Data loaded successfully from Google Sheet
2024-07-05 18:22:05.968569 - INFO - Processing eggshell data
2024-07-05 18:22:06.058944 - INFO - Eggshell data processed successfully
   temp (°C)   time  Asx_Conc_FAA_Mean  Asx_Conc_FAA_Std  Asx_Conc_FAA_Count  \
0         80    0.0          43.333333          8.082904                   3   
1         80   24.0          83.250000         35.453021                   4   
2         80   96.0         161.000000          0.000000                   1   
3         80   97.3          67.333333         15.044379                   3   
4         80  120.0         171.714286         18.034359                   7   

   Glx_Conc_FAA_Mean  Glx_Conc_FAA_Std  Glx_Conc_FAA_Count  Ser_Conc_FAA_Mean  \
0          25.666667          5.033223                   3         166.000000   
1          52.000000         21.400935                   4         163.250000   
2          95.000000          0.000000                   1     

<ipython-input-28-b6148bff994b>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=rename_mapping, inplace=True)
<ipython-input-28-b6148bff994b>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')


In [26]:
#@title ModulAAR Core
class ModulAAR:
    def __init__(self, config):
        self.config = config
        self.data = None
        self.processed_data = None
        self.logger = self._setup_logger()

    def _setup_logger(self):
        from datetime import datetime
        return lambda msg, level="INFO": print(f"{datetime.now()} - {level} - {msg}")

    def load_data(self, source, is_gsheet=False):
        """Load data from a CSV file or Google Sheets."""
        try:
            if is_gsheet:
                sheet_id = source['sheet_id']
                gid = source['gid']
                export_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
                self.data = pd.read_csv(export_url)
                self.logger(f"Data loaded successfully from Google Sheet")
            else:
                self.data = pd.read_csv(source)
                self.logger(f"Data loaded successfully from {source}")
        except Exception as e:
            self.logger(f"Error loading data: {str(e)}", "ERROR")
            self.data = None
        return self.data

    def process_data(self, df=None):
        """Process the loaded data based on its type."""
        if df is None:
            df = self.data
        if df is None:
            self.logger("No data available for processing.", "ERROR")
            return None

        df = df.dropna()

        expected_eggshell_columns = [
            'Pre-heat bleach time (h)', 'temp (°C)', 'pH', 'sample', 'time',
            '[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]',
            'Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L'
        ]
        expected_betalactoglobulin_columns = ['Condition', 'Fraction', 'm']

        if all(col in df.columns for col in expected_eggshell_columns):
            self.logger("Processing eggshell data")
            return self._process_eggshell_data(df)
        elif all(col in df.columns for col in expected_betalactoglobulin_columns):
            self.logger("Processing beta-lactoglobulin data")
            return self._process_betalactoglobulin_data(df)
        else:
            self.logger(
                f"Unknown data format. Expected columns for eggshell: {expected_eggshell_columns}, "
                f"or beta-lactoglobulin: {expected_betalactoglobulin_columns}. Found columns: {list(df.columns)}",
                "ERROR"
            )
            return None

    def _process_eggshell_data(self, df):
        """Process eggshell data."""
        df = self.clean_data(df)
        grouped_faa, grouped_thaa = self.calculate_stats(df)

        processed_df = pd.merge(grouped_faa, grouped_thaa, on=['temp (°C)', 'time'], how='outer')

        processed_df['temp (K)'] = processed_df['temp (°C)'] + 273.15

        self.logger("Eggshell data processed successfully")
        return processed_df

    def _process_betalactoglobulin_data(self, df):
        """Process beta-lactoglobulin data."""
        relevant_columns = ['Condition', 'Fraction', 'm', '[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]',
                            'Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L']
        df = df[relevant_columns]

        # Convert concentration and D/L columns to numeric
        for col in df.columns:
            if col.startswith('[') or col.endswith('D/L'):
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Group by condition and calculate statistics
        grouped = df.groupby(['Condition', 'Fraction', 'm']).agg({
            col: ['mean', 'std'] for col in df.columns if col not in ['Condition', 'Fraction', 'm']
        }).reset_index()

        # Flatten column names
        grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

        self.logger("Beta-lactoglobulin data processed successfully")
        return grouped

    def clean_data(self, df):
        """Clean the eggshell data by converting to numeric and filtering."""
        concentration_columns = ['[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]']
        dl_columns = ['Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L']

        for col in concentration_columns + dl_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        df = df.dropna(subset=concentration_columns + dl_columns)
        return df

    def calculate_stats(self, df):
        """Calculate statistics for FAA and THAA samples."""
        faa_df = df[df['sample'] == 'FAA']
        thaa_df = df[df['sample'] == 'THAA']

        grouped_faa = faa_df.groupby(['temp (°C)', 'time']).agg({
            '[Asx]': ['mean', 'std', 'count'],
            '[Glx]': ['mean', 'std', 'count'],
            '[Ser]': ['mean', 'std', 'count'],
            '[Ala]': ['mean', 'std', 'count'],
            '[Val]': ['mean', 'std', 'count'],
            '[Phe]': ['mean', 'std', 'count'],
            '[Ile]': ['mean', 'std', 'count'],
            'Asx D/L': ['mean', 'std', 'count'],
            'Glx D/L': ['mean', 'std', 'count'],
            'Ser D/L': ['mean', 'std', 'count'],
            'Ala D/L': ['mean', 'std', 'count'],
            'Val D/L': ['mean', 'std', 'count'],
            'Phe D/L': ['mean', 'std', 'count'],
            'Ile D/L': ['mean', 'std', 'count']
        }).reset_index()

        grouped_thaa = thaa_df.groupby(['temp (°C)', 'time']).agg({
            '[Asx]': ['mean', 'std', 'count'],
            '[Glx]': ['mean', 'std', 'count'],
            '[Ser]': ['mean', 'std', 'count'],
            '[Ala]': ['mean', 'std', 'count'],
            '[Val]': ['mean', 'std', 'count'],
            '[Phe]': ['mean', 'std', 'count'],
            '[Ile]': ['mean', 'std', 'count'],
            'Asx D/L': ['mean', 'std', 'count'],
            'Glx D/L': ['mean', 'std', 'count'],
            'Ser D/L': ['mean', 'std', 'count'],
            'Ala D/L': ['mean', 'std', 'count'],
            'Val D/L': ['mean', 'std', 'count'],
            'Phe D/L': ['mean', 'std', 'count'],
            'Ile D/L': ['mean', 'std', 'count']
        }).reset_index()

        new_columns_faa = ['temp (°C)', 'time'] + [
            f'{aa}_Conc_FAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']] + [
            f'{aa}_D/L_FAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']]
        new_columns_thaa = ['temp (°C)', 'time'] + [
            f'{aa}_Conc_THAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']] + [
            f'{aa}_D/L_THAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']]

        grouped_faa.columns = new_columns_faa
        grouped_thaa.columns = new_columns_thaa

        for col in new_columns_faa:
            if '_Std' in col:
                count_col = col.replace('_Std', '_Count')
                grouped_faa.loc[grouped_faa[count_col] == 1, col] = 0

        for col in new_columns_thaa:
            if '_Std' in col:
                count_col = col.replace('_Std', '_Count')
                grouped_thaa.loc[grouped_thaa[count_col] == 1, col] = 0

        return grouped_faa, grouped_thaa

    def calculate_amino_acid_distribution(self, df, amino_acid):
        """Calculate the distribution of a given amino acid."""
        if df is None:
            self.logger("No processed data available. Please process data first.", "ERROR")
            return None

        amino_acid_distributions = []

        initial_thaa = df[f"{amino_acid}_Conc_THAA_Mean"].iloc[0]

        for index, row in df.iterrows():
            thaa = row[f"{amino_acid}_Conc_THAA_Mean"]
            faa = row[f"{amino_acid}_Conc_FAA_Mean"]
            thaa_dl = row[f"{amino_acid}_D/L_THAA_Mean"]
            faa_dl = row[f"{amino_acid}_D/L_FAA_Mean"]
            time_point = row["time"]

            # Calculate BAA, FAA, and losses
            baa = thaa - faa
            baa_d = baa * thaa_dl / (1 + thaa_dl)
            baa_l = baa - baa_d
            faa_d = faa * faa_dl / (1 + faa_dl)
            faa_l = faa - faa_d
            total_loss = max(initial_thaa - thaa, 0)
            loss_d = total_loss * faa_dl / (1 + faa_dl) if total_loss > 0 else 0
            loss_l = total_loss - loss_d if total_loss > 0 else 0

            # Calculate standard deviations
            thaa_std = row[f"{amino_acid}_Conc_THAA_Std"]
            faa_std = row[f"{amino_acid}_Conc_FAA_Std"]
            baa_std = np.sqrt(thaa_std**2 + faa_std**2)
            baa_d_std = baa_std * thaa_dl / (1 + thaa_dl)
            baa_l_std = baa_std - baa_d_std
            faa_d_std = faa_std * faa_dl / (1 + faa_dl)
            faa_l_std = faa_std - faa_d_std
            loss_std = np.sqrt(2 * thaa_std**2)
            loss_d_std = loss_std * faa_dl / (1 + faa_dl)
            loss_l_std = loss_std - loss_d_std

            amino_acid_distributions.append({
                "time": time_point,
                "temp (K)": row["temp (K)"],
                "BAA_D": baa_d, "BAA_L": baa_l,
                "BAA_D_Std": baa_d_std, "BAA_L_Std": baa_l_std,
                "FAA_D": faa_d, "FAA_L": faa_l,
                "FAA_D_Std": faa_d_std, "FAA_L_Std": faa_l_std,
                "FAA_D_loss": loss_d, "FAA_L_loss": loss_l,
                "FAA_D_loss_Std": loss_d_std, "FAA_L_loss_Std": loss_l_std,
            })

        return pd.DataFrame(amino_acid_distributions)

# Usage
config = {
    'N': 50000,
    'fold_water': 8,
    'temperature_kelvin': 353.15,
    'amino_acid': 'Asx',
    'initial_length': 1500,
    'rate_params': {
        # Add rate parameters here
    }
}

model = ModulAAR(config)

# Load data from Google Sheets
sheet_info = {'sheet_id': "1nA6jSAkAf1Ud-kHdaYTMtBPgKhe9nBg_IjM9idLlj8E", 'gid': "1259514505"}
df1 = model.load_data(sheet_info, is_gsheet=True)

# Process data
if df1 is not None:
    processed_df1 = model.process_data(df1)
    if processed_df1 is not None:
        print(processed_df1.head())

        # Calculate amino acid distribution for Asx
        asx_distribution = model.calculate_amino_acid_distribution(processed_df1, 'Asx')
        print(asx_distribution.head())


2024-07-05 18:08:25.235547 - INFO - Data loaded successfully from Google Sheet
2024-07-05 18:08:25.238211 - ERROR - Unknown data format. Expected columns for eggshell: ['Pre-heat bleach time (h)', 'temp (°C)', 'pH', 'sample', 'time', '[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]', 'Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L'], or beta-lactoglobulin: ['Condition', 'Fraction', 'm']. Found columns: ['Pre-heat bleach time (h)', 'temp (°C)', 'pH', 'sample', 'time (h)', '[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]', 'Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L']


###The Old stuff


---



In [ ]:
#@title Global Variables and Constants
# Precomputed activation energies in kJ/mol (converted to J/mol)
activation_energies = {
    'Asx_ConcTHAA': 111.09 * 1000, 'Asx_ConcFAA': 98.93 * 1000, 'Asx_DLTHAA': 126.02 * 1000, 'Asx_DLFAA': 121.99 * 1000,
    'Glx_ConcTHAA': 105.46 * 1000, 'Glx_ConcFAA': 109.39 * 1000, 'Glx_DLTHAA': 121.89 * 1000, 'Glx_DLFAA': 100.80 * 1000,
    'Ser_ConcTHAA': 127.59 * 1000, 'Ser_ConcFAA': 97.95 * 1000, 'Ser_DLTHAA': 123.03 * 1000, 'Ser_DLFAA': 116.72 * 1000,
    'Ala_ConcTHAA': 96.58 * 1000, 'Ala_ConcFAA': 112.12 * 1000, 'Ala_DLTHAA': 113.58 * 1000, 'Ala_DLFAA': 98.88 * 1000,
    'Val_ConcTHAA': 111.00 * 1000, 'Val_ConcFAA': 122.81 * 1000, 'Val_DLTHAA': 99.73 * 1000, 'Val_DLFAA': 128.52 * 1000,
    'Phe_ConcTHAA': 113.28 * 1000, 'Phe_ConcFAA': 98.88 * 1000, 'Phe_DLTHAA': 128.15 * 1000, 'Phe_DLFAA': 102.27 * 1000,
    'Leu_ConcTHAA': 114.09 * 1000, 'Leu_ConcFAA': 103.75 * 1000, 'Leu_DLTHAA': 105.43 * 1000, 'Leu_DLFAA': 105.43 * 1000,
    'Ile_ConcTHAA': 114.09 * 1000, 'Ile_ConcFAA': 103.75 * 1000, 'Ile_DLTHAA': 105.43 * 1000, 'Ile_DLFAA': 105.43 * 1000
}

# Mapping from one-letter to three-letter amino acid codes
one_to_three_letter = {
    "I": "Ile", "V": "Val", "L": "Leu", "F": "Phe", "C": "Cys", "M": "Met",
    "A": "Ala", "G": "Gly", "T": "Thr", "W": "Trp", "S": "Ser", "Y": "Tyr",
    "P": "Pro", "H": "His", "Q": "Gln", "E": "Glx", "N": "Asn", "D": "Asx",
    "K": "Lys", "R": "Arg", "D": "Asx", "E": "Glx"
}

# Mapping three-letter codes to concentration column names
three_letter_to_Conc = {
    'Ala': '[Ala]', 'Arg': '[Arg]', 'Asn': '[Asx]', 'Asp': '[Asx]', 'Cys': '[Cys]',
    'Glu': '[Glx]', 'Gln': '[Glx]', 'Gly': '[Gly]', 'His': '[His]', 'Ile': '[Ile]',
    'Leu': '[Leu]', 'Lys': '[Lys]', 'Met': '[Met]', 'Phe': '[Phe]', 'Pro': '[Pro]',
    'Ser': '[Ser]', 'Thr': '[Thr]', 'Trp': '[Trp]', 'Tyr': '[Tyr]', 'Val': '[Val]'
}


# Pre-exponential factors for each type of rate
pre_exponential_factors = {
    'Asx': {'internal': 1e19, 'terminal': 1e20, 'free': 1e18, 'loss': 1e18},
    'Glx': {'internal': 1e17, 'terminal': 1e20, 'free': 1e17, 'loss': 1e16},
    'Ser': {'internal': 1e19, 'terminal': 1e20, 'free': 1e20, 'loss': 1e15},
    'Ala': {'internal': 1e17, 'terminal': 1e17, 'free': 1e16, 'loss': 1e20},
    'Val': {'internal': 1e17, 'terminal': 1e20, 'free': 1e15, 'loss': 1e10},
    'Phe': {'internal': 1e17, 'terminal': 1e17, 'free': 1e20, 'loss': 1e15},
    'Leu': {'internal': 1e19, 'terminal': 1e18, 'free': 1e13, 'loss': 1e20},
    'Ile': {'internal': 1e16, 'terminal': 1e20, 'free': 1e18, 'loss': 1e20}
}

# Mapping from rate type descriptions to pre-exponential factor keys
rate_type_mapping = {
    'ConcTHAA': 'internal',
    'ConcFAA': 'terminal',
    'DLTHAA': 'free',
    'DLFAA': 'loss'
}

# Universal gas constant in J/(mol·K)
R = 8.314

In [ ]:
#@title Helper Functions
def arrhenius_rate_constant(E_a, T, A):
    return A * np.exp(-E_a / (R * T))

def adjust_saturation(color, amount=0.5):
    color = mcolors.to_rgba(color)
    color = mcolors.rgb_to_hsv(color[:3])
    color[1] = max(0, min(1, color[1] * amount))
    return mcolors.hsv_to_rgb(color)

In [ ]:
#@title Color Setup
# Load color dictionary
color_dict_path = '/content/drive/MyDrive/Colab_Notebooks/Dictionaries/Colours/colors.json'
with open(color_dict_path, 'r') as file:
    color_dict = json.load(file)

amino_acid_colors = color_dict["amino_acids_colors"]

# Create dictionaries for three-letter codes and concentration column names
amino_acid_colors_three_letter = {one_to_three_letter[k]: v for k, v in amino_acid_colors.items() if k in one_to_three_letter}
three_letter_to_Conc = {k: f'[{k}]' for k in amino_acid_colors_three_letter.keys()}
amino_acid_colors_conc = {three_letter_to_Conc[k]: v for k, v in amino_acid_colors_three_letter.items() if k in three_letter_to_Conc}

# Adjust colors for standard deviations
std_colors = {k: adjust_saturation(v, 0.5) for k, v in amino_acid_colors_three_letter.items()}


In [ ]:
#@title Fixed Variables and Amino Acid Selection
NUM_TIME_POINTS = 48
amino_acid = 'Asx'
initial_length = 1500
temperature_kelvin = 353.15

In [ ]:
#@title #Load Data from Google Sheets
sheet_id = "1nA6jSAkAf1Ud-kHdaYTMtBPgKhe9nBg_IjM9idLlj8E"
gid = "1259514505"
export_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"

try:
    df = pd.read_csv(export_url)
    print("Data successfully loaded into DataFrame!")
except Exception as e:
    print(f"Error: Could not access data. {e}")

# Ensure the columns match your Google Sheets structure
expected_columns = [
    'Pre-heat bleach time (h)', 'temp (°C)', 'pH', 'sample', 'time',
    '[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]',
    'Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L'
]

# Rename the column (assuming extracted time or existing time format)
df = df.rename(columns={'time (h)': 'time'})

# Ensure all expected columns are present in the data
data = df[expected_columns]


In [ ]:
#@title Data Processing Functions
def clean_data(df):
    # Convert concentration and D/L columns to numeric, coerce errors to NaN
    concentration_columns = ['[Asx]', '[Glx]', '[Ser]', '[Ala]', '[Val]', '[Phe]', '[Ile]']
    dl_columns = ['Asx D/L', 'Glx D/L', 'Ser D/L', 'Ala D/L', 'Val D/L', 'Phe D/L', 'Ile D/L']

    for col in concentration_columns + dl_columns:
        df.loc[:, col] = pd.to_numeric(df[col], errors='coerce')

    # Filter out rows with comments or non-numeric entries in critical columns
    df = df.dropna(subset=concentration_columns + dl_columns)

    return df

def calculate_stats(df):
    faa_df = df[df['sample'] == 'FAA']
    thaa_df = df[df['sample'] == 'THAA']

    grouped_faa = faa_df.groupby(['temp (°C)', 'time']).agg({
        '[Asx]': ['mean', 'std', 'count'],
        '[Glx]': ['mean', 'std', 'count'],
        '[Ser]': ['mean', 'std', 'count'],
        '[Ala]': ['mean', 'std', 'count'],
        '[Val]': ['mean', 'std', 'count'],
        '[Phe]': ['mean', 'std', 'count'],
        '[Ile]': ['mean', 'std', 'count'],
        'Asx D/L': ['mean', 'std', 'count'],
        'Glx D/L': ['mean', 'std', 'count'],
        'Ser D/L': ['mean', 'std', 'count'],
        'Ala D/L': ['mean', 'std', 'count'],
        'Val D/L': ['mean', 'std', 'count'],
        'Phe D/L': ['mean', 'std', 'count'],
        'Ile D/L': ['mean', 'std', 'count']
    }).reset_index()

    grouped_thaa = thaa_df.groupby(['temp (°C)', 'time']).agg({
        '[Asx]': ['mean', 'std', 'count'],
        '[Glx]': ['mean', 'std', 'count'],
        '[Ser]': ['mean', 'std', 'count'],
        '[Ala]': ['mean', 'std', 'count'],
        '[Val]': ['mean', 'std', 'count'],
        '[Phe]': ['mean', 'std', 'count'],
        '[Ile]': ['mean', 'std', 'count'],
        'Asx D/L': ['mean', 'std', 'count'],
        'Glx D/L': ['mean', 'std', 'count'],
        'Ser D/L': ['mean', 'std', 'count'],
        'Ala D/L': ['mean', 'std', 'count'],
        'Val D/L': ['mean', 'std', 'count'],
        'Phe D/L': ['mean', 'std', 'count'],
        'Ile D/L': ['mean', 'std', 'count']
    }).reset_index()

    # Rename columns to reflect FAA and THAA
    new_columns_faa = ['temp (°C)', 'time'] + [f'{aa}_Conc_FAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']] + \
                      [f'{aa}_D/L_FAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']]
    new_columns_thaa = ['temp (°C)', 'time'] + [f'{aa}_Conc_THAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']] + \
                       [f'{aa}_D/L_THAA_{stat}' for aa in ['Asx', 'Glx', 'Ser', 'Ala', 'Val', 'Phe', 'Ile'] for stat in ['Mean', 'Std', 'Count']]

    grouped_faa.columns = new_columns_faa
    grouped_thaa.columns = new_columns_thaa

    for col in new_columns_faa:
        if '_Std' in col:
            count_col = col.replace('_Std', '_Count')
            grouped_faa.loc[grouped_faa[count_col] == 1, col] = 0

    for col in new_columns_thaa:
        if '_Std' in col:
            count_col = col.replace('_Std', '_Count')
            grouped_thaa.loc[grouped_thaa[count_col] == 1, col] = 0

    return grouped_faa, grouped_thaa

def prepare_and_filter_data(df, amino_acid, temperature_kelvin):
    relevant_suffixes = ['Mean', 'Std', 'DL_Mean', 'DL_Std']
    relevant_columns = [
        col for col in df.columns
        if amino_acid in col and any(suffix in col for suffix in relevant_suffixes)
    ]
    relevant_columns.extend(['time', 'temp (K)'])

    filtered_data = df[
        (df['temp (K)'] == temperature_kelvin)
    ][relevant_columns]

    return filtered_data

def calculate_amino_acid_distribution_with_loss(df, amino_acid):
    amino_acid_distributions = []

    initial_thaa = df[f"{amino_acid}_Conc_THAA_Mean"].iloc[0]

    for index, row in df.iterrows():
        thaa = row[f"{amino_acid}_Conc_THAA_Mean"]
        faa = row[f"{amino_acid}_Conc_FAA_Mean"]
        thaa_dl = row[f"{amino_acid}_D/L_THAA_Mean"]
        faa_dl = row[f"{amino_acid}_D/L_FAA_Mean"]
        time_point = row["time"]

        # Calculate BAA
        baa = thaa - faa
        baa_d = baa * thaa_dl / (1 + thaa_dl)
        baa_l = baa - baa_d

        # Calculate FAA
        faa_d = faa * faa_dl / (1 + faa_dl)
        faa_l = faa - faa_d

        # Calculate total loss
        total_loss = max(initial_thaa - thaa, 0)

        # Distribute total loss between D and L forms based on current FAA D/L ratio
        if total_loss > 0:
            loss_d = total_loss * faa_dl / (1 + faa_dl)
            loss_l = total_loss - loss_d
        else:
            loss_d = 0
            loss_l = 0

        # Calculate standard deviations
        thaa_std = row[f"{amino_acid}_Conc_THAA_Std"] if f"{amino_acid}_Conc_THAA_Std" in df.columns else 0
        faa_std = row[f"{amino_acid}_Conc_FAA_Std"] if f"{amino_acid}_Conc_FAA_Std" in df.columns else 0

        baa_std = np.sqrt(thaa_std**2 + faa_std**2)
        baa_d_std = baa_std * thaa_dl / (1 + thaa_dl)
        baa_l_std = baa_std - baa_d_std

        faa_d_std = faa_std * faa_dl / (1 + faa_dl)
        faa_l_std = faa_std - faa_d_std

        loss_std = np.sqrt(2 * thaa_std**2)  # Using THAA std for loss calculation
        loss_d_std = loss_std * faa_dl / (1 + faa_dl)
        loss_l_std = loss_std - loss_d_std

        amino_acid_distribution = {
            "time": time_point,
            "temp (K)": row["temp (K)"],
            "BAA_D": baa_d,
            "BAA_L": baa_l,
            "BAA_D_Std": baa_d_std,
            "BAA_L_Std": baa_l_std,
            "FAA_D": faa_d,
            "FAA_L": faa_l,
            "FAA_D_Std": faa_d_std,
            "FAA_L_Std": faa_l_std,
            "FAA_D_loss": loss_d,
            "FAA_L_loss": loss_l,
            "FAA_D_loss_Std": loss_d_std,
            "FAA_L_loss_Std": loss_l_std,
        }

        amino_acid_distributions.append(amino_acid_distribution)

    return pd.DataFrame(amino_acid_distributions)


In [ ]:
#@title Data Processing and Preparation
# Filter and clean data
cleaned_data = clean_data(data)
print(" ")
print("cleaned_data:--------------------------")
print(cleaned_data.head(3))

# Calculate statistics
faa_stats, thaa_stats = calculate_stats(cleaned_data)

# Merge FAA and THAA statistics
data_interpolated = pd.merge(faa_stats, thaa_stats, on=['temp (°C)', 'time'], how='outer')

# Add 'temp (K)' column to the merged DataFrame
data_interpolated['temp (K)'] = data_interpolated['temp (°C)'] + 273.15

# Calculate BAA
real_data = prepare_and_filter_data(data_interpolated, amino_acid, temperature_kelvin)

print("real_data:--------------------------")
print(real_data.head(3))

# Check prepared data
real_DL = calculate_amino_acid_distribution_with_loss(real_data, amino_acid)

# Print the DataFrame
print(" ")
print("real_DL:--------------------------")
print(real_DL.head(2))


In [ ]:
#@title Water Generation Calculation Functions
def categorize_amino_acids(df):
    concentration_columns = [col for col in df.columns if col.startswith('[') and col.endswith(']')]
    categories = {
        'THAA_dehyd': ['[Ala]'],  # Alanine for dehydration
        'THAA_cyclic': ['[Glx]'],
        'THAA_serine': ['[Ser]'],  # Separate category for Serine
        'THAA_no_water': [col for col in concentration_columns if col not in ['[Ala]', '[Glx]', '[Ser]']]
    }
    return categories

def initialize_concentrations(df, categories):
    concentrations = {}

    for category, amino_acids in categories.items():
        category_dict = {}
        for aa in amino_acids:
            stats = df.groupby('time')[aa].agg(['mean', 'std']).reset_index()
            category_dict[aa] = stats
        concentrations[category] = category_dict

    return concentrations

def average_initial_concentration(data, column, num_points=3):
    return data[column].iloc[:num_points].mean()

def calculate_changes_in_concentration(data, initial_alanine, initial_serine):
    time_points = data['time'].values
    alanine_concentration = data['[Ala]'].values
    serine_concentration = data['[Ser]'].values

    change_in_alanine = alanine_concentration - initial_alanine
    change_in_serine = serine_concentration - initial_serine

    return time_points, change_in_alanine, change_in_serine

def linear_fit(t, A, B):
    return A * t + B

def exponential_decay(t, A, k):
    return A * np.exp(-k * t)

def fit_linear_model(time_points, concentration):
    popt, pcov = curve_fit(linear_fit, time_points, concentration['mean'], sigma=concentration['std'], absolute_sigma=True)
    return popt, pcov

def calculate_r_squared(y_true, y_pred):
    residuals = y_true - y_pred
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

def calculate_water_generation(concentrations, time_points, categories):
    water_generation = np.zeros(len(time_points))

    for aa in categories['THAA_dehyd']:
        initial_concentration = concentrations['THAA_dehyd'][aa]['mean'].iloc[0]
        time_values = concentrations['THAA_dehyd'][aa]['time'].values
        concentration_values = concentrations['THAA_dehyd'][aa]['mean'].values

        time_values = pd.to_numeric(time_values, errors='coerce')
        concentration_values = pd.to_numeric(concentration_values, errors='coerce')

        valid_indices = ~np.isnan(time_values) & ~np.isnan(concentration_values)
        time_values = time_values[valid_indices]
        concentration_values = concentration_values[valid_indices]

        concentration_values = np.interp(time_points, time_values, concentration_values)
        for i, t in enumerate(time_points):
            current_concentration = concentration_values[i]
            water_generation[i] += max(initial_concentration - current_concentration, 0)

    return water_generation

def calculate_cyclic_water(concentrations, time_points, categories):
    cyclic_water = np.zeros(len(time_points))

    for i, t in enumerate(time_points):
        for aa in categories['THAA_cyclic']:
            initial_concentration = concentrations['THAA_cyclic'][aa]['mean'].iloc[0]
            final_concentration = concentrations['THAA_cyclic'][aa]['mean'].iloc[-1]
            cyclic_water[i] = max(initial_concentration - (initial_concentration - final_concentration) * (t / time_points[-1]), 0)

    return cyclic_water

def process_and_plot(cleaned_data, initial_length, amino_acid_colors):
    categories = categorize_amino_acids(cleaned_data)
    concentrations = initialize_concentrations(cleaned_data, categories)
    initial_alanine_concentration = average_initial_concentration(cleaned_data, '[Ala]')
    initial_serine_concentration = average_initial_concentration(cleaned_data, '[Ser]')
    time_points, change_in_alanine, change_in_serine = calculate_changes_in_concentration(
        cleaned_data, initial_alanine_concentration, initial_serine_concentration)

    alanine_stats = cleaned_data.groupby('time')['[Ala]'].agg(['mean', 'std']).reset_index()
    serine_stats = cleaned_data.groupby('time')['[Ser]'].agg(['mean', 'std']).reset_index()

    valid_indices_alanine = np.isin(time_points, alanine_stats['time'])
    filtered_time_points_alanine = time_points[valid_indices_alanine]
    filtered_change_in_alanine = change_in_alanine[valid_indices_alanine]
    filtered_alanine_stats = alanine_stats[alanine_stats['time'].isin(filtered_time_points_alanine)]

    alanine_params, alanine_cov = fit_linear_model(filtered_alanine_stats['time'].values, filtered_alanine_stats)

    valid_indices_serine = np.isin(time_points, serine_stats['time'])
    filtered_time_points_serine = time_points[valid_indices_serine]
    filtered_change_in_serine = change_in_serine[valid_indices_serine]
    filtered_serine_stats = serine_stats[serine_stats['time'].isin(filtered_time_points_serine)]

    serine_params, serine_cov = fit_linear_model(filtered_serine_stats['time'].values, filtered_serine_stats)

    alanine_fit = linear_fit(filtered_time_points_alanine, *alanine_params)
    serine_fit = linear_fit(filtered_time_points_serine, *serine_params)
    alanine_r_squared = calculate_r_squared(filtered_change_in_alanine, alanine_fit)
    serine_r_squared = calculate_r_squared(filtered_change_in_serine, serine_fit)

    dehydration_rate = serine_params[0]
    print(f"Dehydration Rate: k = {dehydration_rate:.4f}")

    water_generation = calculate_water_generation(concentrations, time_points, categories)
    cyclic_water = calculate_cyclic_water(concentrations, time_points, categories)
    cumulative_water_release = np.cumsum(water_generation + cyclic_water)

    fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(50, 10))

    unique_times = cleaned_data['time'].unique()
    bottom = np.zeros(len(unique_times))

    for category in concentrations.values():
        for aa, stats in category.items():
            concentration = pd.to_numeric(stats['mean'].values, errors='coerce')
            valid_indices = ~np.isnan(concentration)
            concentration = concentration[valid_indices]
            color = amino_acid_colors_conc.get(aa, 'gray')
            ax1.fill_between(unique_times[valid_indices], bottom[valid_indices], bottom[valid_indices] + concentration, label=aa, color=color)
            bottom[valid_indices] += concentration

    ax1.set_xlabel('Time (hours)')
    ax1.set_ylabel('Concentration')
    ax1.set_title('Amino Acid Concentrations Over Time')
    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    ala_color = amino_acid_colors_conc.get('[Ala]', 'blue')
    ser_color = amino_acid_colors_conc.get('[Ser]', 'green')
    glx_color = amino_acid_colors_conc.get('[Glx]', 'red')

    ax1.semilogx(time_points, water_generation, label='Water Generation (Ala)', color=ala_color)
    ax1.semilogx(time_points, cyclic_water, label='Cyclic Water (Glx)', color=glx_color)
    ax1.set_xlabel('Time (hours)')
    ax1.set_ylabel('Water Concentration')
    ax1.set_title('Water Generation and Cyclic Water Over Time')
    ax1.legend()

    ax2.semilogx(time_points, water_generation, label='Water Generation (Ala)', color=ala_color)
    ax2.semilogx(time_points, cyclic_water, label='Cyclic Water (Glx)', color=glx_color)
    ax2.set_xlabel('Time (hours)')
    ax2.set_ylabel('Water Concentration')
    ax2.set_title('Water Generation and Cyclic Water Over Time')
    ax2.legend()

    ax3.semilogx(filtered_time_points_alanine, filtered_change_in_alanine, 'o', label='Observed Decline in Alanine', color=ala_color)
    ax3.semilogx(filtered_time_points_alanine, alanine_fit, '-', label=f'Fitted Alanine: A={alanine_params[0]:.2f}, B={alanine_params[1]:.2f}, $R^2$={alanine_r_squared:.2f}', color=ala_color)
    ax3.semilogx(filtered_time_points_serine, filtered_change_in_serine, 'o', label='Observed Decline in Serine', color=ser_color)
    ax3.semilogx(filtered_time_points_serine, serine_fit, '-', label=f'Fitted Serine: A={serine_params[0]:.2f}, B={serine_params[1]:.2f}, $R^2$={serine_r_squared:.2f}', color=ser_color)
    ax3.set_xlabel('Time (hours)')
    ax3.set_ylabel('Decline in Concentration')
    ax3.set_title('Decline in Alanine and Serine Concentrations Over Time')
    ax3.legend()

    ax4.plot(time_points, water_generation, 'o', label='Observed Water Generation', color=ala_color)
    ax4.plot(time_points, water_generation, '-', label='Fitted Water Generation', color=ala_color)
    ax4.plot(time_points, cyclic_water, 'o', label='Observed Cyclic Water', color=glx_color)
    ax4.plot(time_points, cyclic_water, '-', label='Fitted Cyclic Water', color=glx_color)
    ax4.set_xlabel('Time (hours)')
    ax4.set_ylabel('Water Concentration')
    ax4.set_title('Observed and Fitted Water Generation and Cyclic Water')
    ax4.legend()

    ax5.plot(time_points, cumulative_water_release, 'o-', label='Cumulative Water Release', color='purple')
    ax5.set_xlabel('Time (hours)')
    ax5.set_ylabel('Cumulative Water Molecules')
    ax5.set_title('Cumulative Water Release Over Time')
    ax5.legend()

    plt.tight_layout()
    plt.show()
    return dehydration_rate

def adjusted_rate_for_polymer_length(rate, initial_length):
    return rate / initial_length

In [ ]:
#@title Process and plot water generation
dehydration_rate = process_and_plot(cleaned_data, initial_length, amino_acid_colors)
dehydration_rate_adjusted = adjusted_rate_for_polymer_length(dehydration_rate, initial_length)
print(f"Adjusted Serine Dehydration Rate: k = {dehydration_rate_adjusted:.4f}")


In [ ]:
#@title Simulation Functions
def second_order_hydrolysis(peptide_conc, water_conc, rate_constant, dt):
    effective_rate = rate_constant * water_conc * dt
    return peptide_conc * effective_rate / (1 + effective_rate)
#@title Updated Simulation Functions

def simulate_AAR(state, rates, dehydration_rate):
    new_state = state.copy()

    # 2nd order Hydrolysis
    for source, target in [('int', 'term'), ('term', 'free')]:
        if source == 'int':
            total_fast = state['L_int_fast'] + state['D_int_fast']
            total_slow = state['L_int_slow'] + state['D_int_slow']
            total = total_fast + total_slow
        else:
            total = state[f'L_{source}'] + state[f'D_{source}']

        if total > 1 and state['water'] > 0:  # Stop hydrolysis if total is 1 or less
            effective_rate = rates[f'k_{source}'] * (state['water'] / (N * fold_water))

            if source == 'int':
                hydrolyzed_fast = np.random.binomial(total_fast, effective_rate)
                hydrolyzed_slow = np.random.binomial(total_slow, rates['slow_hydrolysis_pool_relative_rate'])
                total_hydrolyzed = hydrolyzed_fast + hydrolyzed_slow
            else:
                total_hydrolyzed = np.random.binomial(total, effective_rate)

            for chirality in ['L', 'D']:e
                if source == 'int':
                    if total_fast > 0:
                        moved_fast = int(hydrolyzed_fast * (state[f'{chirality}_{source}_fast'] / total_fast))
                    else:
                        moved_fast = 0

                    if total_slow > 0:
                        moved_slow = int(hydrolyzed_slow * (state[f'{chirality}_{source}_slow'] / total_slow))
                    else:
                        moved_slow = 0

                    new_state[f'{chirality}_{source}_fast'] -= moved_fast
                    new_state[f'{chirality}_{source}_slow'] -= moved_slow
                    new_state[f'{chirality}_{target}'] += moved_fast + moved_slow
                else:
                    moved = int(total_hydrolyzed * (state[f'{chirality}_{source}'] / total))
                    new_state[f'{chirality}_{source}'] -= moved
                    new_state[f'{chirality}_{target}'] += moved

            new_state['water'] -= total_hydrolyzed

    # Racemization
    for location in ['int_fast', 'int_slow', 'term', 'free']:
        if location.startswith('int'):
            total = state[f'L_{location}'] + state[f'D_{location}']
        else:
            total = state[f'L_{location}'] + state[f'D_{location}']

        if total > N * 0.005:  # Stop racemization if total is less than 0.05%
            rate = rates[f'racemization_rate_{location.split("_")[0]}']
            if location == 'int_slow':
                rate = rates['slow_racemization_rate_polymer_rate']

            for source, target in [('L', 'D'), ('D', 'L')]:
                racemized = np.random.binomial(state[f'{source}_{location}'], rate)
                new_state[f'{source}_{location}'] -= racemized
                new_state[f'{target}_{location}'] += racemized

    # Loss of free amino acids
    for chirality in ['L', 'D']:
        lost = np.random.binomial(state[f'{chirality}_free'], rates['k_loss'])
        new_state[f'{chirality}_free'] -= lost

    # Add water back into the simulation
    new_state['water'] += dehydration_rate

    return new_state

def run_simulation(initial_state, rates, time_steps, dehydration_rate):
    results = {key: [value] for key, value in initial_state.items()}
    results['time'] = [0]

    state = initial_state.copy()
    for t in range(1, time_steps + 1):
        state = simulate_AAR(state, rates, dehydration_rate)
        for key, value in state.items():
            results[key].append(value)
        results['time'].append(t)

    return results

def calculate_ratios(results):
    ratios = {}
    for location in ['int_fast', 'int_slow', 'term', 'free']:
        L = np.array(results[f'L_{location}'])
        D = np.array(results[f'D_{location}'])
        ratios[f'{location}_D_L_ratio'] = np.divide(D, L, where=L != 0)

    total_L = sum(np.array(results[f'L_{loc}']) for loc in ['int_fast', 'int_slow', 'term', 'free'])
    total_D = sum(np.array(results[f'D_{loc}']) for loc in ['int_fast', 'int_slow', 'term', 'free'])
    ratios['overall_D_L_ratio'] = np.divide(total_D, total_L, where=total_L != 0)

    return ratios



In [ ]:
#@title plotting siumlation

# Define colormaps using the correct module and method
d_colormap = plt.colormaps['tab20b']
l_colormap = plt.colormaps['tab20b']

# Function to get color based on D/L and lightness
def get_color(is_d, is_predicted, lightness):
    if is_d:
        colormap = d_colormap
    else:
        colormap = l_colormap
    return colormap(lightness)

def plot_results(results, ratios, real_DL):
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))

    # Plot internal amino acids (combining fast and slow fractions)
    ax = axs[0, 0]
    ax.plot(results['time'], np.array(results['L_int_fast']) + np.array(results['L_int_slow']), label='Internal L')
    ax.plot(results['time'], np.array(results['D_int_fast']) + np.array(results['D_int_slow']), label='Internal D')
    ax.set_xlabel('Time')
    ax.set_ylabel('Number of Amino Acids')
    ax.set_title('Internal Amino Acids')
    ax.legend()

    # Plot terminal amino acids
    ax = axs[0, 1]
    ax.plot(results['time'], results['L_term'], label='Terminal L')
    ax.plot(results['time'], results['D_term'], label='Terminal D')
    ax.set_xlabel('Time')
    ax.set_ylabel('Number of Amino Acids')
    ax.set_title('Terminal Amino Acids')
    ax.legend()

    # Plot free amino acids
    ax = axs[0, 2]
    ax.plot(results['time'], results['L_free'], label='Free L')
    ax.plot(results['time'], results['D_free'], label='Free D')
    ax.set_xlabel('Time')
    ax.set_ylabel('Number of Amino Acids')
    ax.set_title('Free Amino Acids')
    ax.legend()

    # Plot D/L ratios
    ax = axs[1, 0]
    ax.plot(results['time'], ratios['int_fast_D_L_ratio'], label='Internal Fast')
    ax.plot(results['time'], ratios['int_slow_D_L_ratio'], label='Internal Slow')
    ax.plot(results['time'], ratios['term_D_L_ratio'], label='Terminal')
    ax.plot(results['time'], ratios['free_D_L_ratio'], label='Free')
    ax.plot(results['time'], ratios['overall_D_L_ratio'], label='Overall', linewidth=2)
    ax.set_xlabel('Time')
    ax.set_ylabel('D/L Ratio')
    ax.set_title('D/L Ratios')
    ax.legend()

    # Plot water consumption
    ax = axs[1, 1]
    ax.plot(results['time'], results['water'], label='Water Molecules')
    ax.set_xlabel('Time')
    ax.set_ylabel('Number of Water Molecules')
    ax.set_title('Water Consumption')
    ax.legend()

    # Plot total amino acids
    ax = axs[1, 2]
    total_amino_acids = (np.array(results['L_int_fast']) + np.array(results['D_int_fast']) +
                         np.array(results['L_int_slow']) + np.array(results['D_int_slow']) +
                         np.array(results['L_term']) + np.array(results['D_term']) +
                         np.array(results['L_free']) + np.array(results['D_free']))
    ax.plot(results['time'], total_amino_acids, label='Total Amino Acids')
    ax.set_xlabel('Time')
    ax.set_ylabel('Number of Amino Acids')
    ax.set_title('Total Amino Acids')
    ax.legend()

    # Plot real_DL experimental data
    axs[0, 0].scatter(real_DL['time'], real_DL['FAA_D'], label='FAA_D (real)', color=get_color(True, False, 0.7))
    axs[0, 0].scatter(real_DL['time'], real_DL['FAA_L'], label='FAA_L (real)', color=get_color(False, False, 0.7))
    axs[0, 0].scatter(real_DL['time'], real_DL['BAA_D'], label='BAA_D (real)', color=get_color(True, False, 0.6))
    axs[0, 0].scatter(real_DL['time'], real_DL['BAA_L'], label='BAA_L (real)', color=get_color(False, False, 0.6))
    axs[0, 0].scatter(real_DL['time'], real_DL['FAA_D_loss'] + real_DL['FAA_L_loss'], label='Loss', color='grey')

    thaa_dl_real = (real_DL['FAA_D'] + real_DL['BAA_D']) / (real_DL['FAA_L'] + real_DL['BAA_L'])
    axs[0, 0].scatter(real_DL['time'], thaa_dl_real, label='THAA D/L (real)', color=get_color(True, False, 0.5))

    plt.tight_layout()
    plt.show()

In [ ]:
#@title Model Parameters
N = 50000  # Initial number of amino acids
fold_water = 8  # Initial water concentration relative to amino acids
k_internal = 0.04  # Rate constant for internal to terminal hydrolysis
k_terminal = 0.01  # Rate constant for terminal to free hydrolysis
k_loss = 0.001  # Rate of free amino acid loss
racemization_rate_polymer = 0.001  # Racemization rate for internal amino acids
racemization_rate_terminal = 0.002  # Racemization rate for terminal amino acids
racemization_rate_free = 0.01  # Racemization rate for free amino acids
time_steps = 1000  # Number of time steps for the simulation
dt = 1  # Time step size
user_defined_max_time = 6000  # Example of user-defined max_time
# Set the number of time steps
number_of_steps = 1000

# New parameters for slow fractions
slow_internal_hydrolysis_fraction = 0.5
slow_internal_hydrolysis_rate = 0.01
slow_racemization_rate_polymer_fraction = 0.5
slow_racemization_rate_polymer_rate = 0.001

# Initial conditions
initial_state = {
    'L_int_fast': int(N * (1 - slow_internal_hydrolysis_fraction)),
    'L_int_slow': int(N * slow_internal_hydrolysis_fraction),
    'D_int_fast': 0,
    'D_int_slow': 0,
    'L_term': 0,
    'D_term': 0,
    'L_free': 0,
    'D_free': 0,
    'water': N * fold_water  # Initial water molecules
}

rate_params = {
    'k_internal': 0.04,
    'k_terminal': 0.02,
    'k_loss': 0.001,
    'fixed_rate_free': 0.005,
    'slow_hydrolysis_pool_size': 0.5,
    'slow_hydrolysis_pool_relative_rate': 0.00013,
    'slow_DL_terminal_pool_size': 0.5,
    'slow_DL_terminal_pool_relative_rate': 0.1,
    'slow_DL_internal_pool_size': 0.34762483392850024,
    'racemization_rate_polymer': 0.001,
    'racemization_rate_terminal': 0.002,
    'racemization_rate_free': 0.01,
}

rates = {
    'k_int': k_internal * dt,
    'k_term': k_terminal * dt,
    'k_loss': k_loss * dt,
    'racemization_rate_int': racemization_rate_polymer * dt,
    'racemization_rate_term': racemization_rate_terminal * dt,
    'racemization_rate_free': racemization_rate_free * dt,
    'slow_hydrolysis_pool_relative_rate': slow_internal_hydrolysis_rate * dt,
    'slow_racemization_rate_polymer_rate': slow_racemization_rate_polymer_rate * dt
}

In [ ]:
#@title Run Updated Simulation



# Calculate the maximum time
max_time = min(real_DL['time'].max(), user_defined_max_time)

# Calculate the time step size
dt = max_time / number_of_steps

# Update rate parameters
rate_params['k_loss'] = serine_dehydration_rate_adjusted

# Convert rate parameters to rates used in simulation
rates = {key: value * dt for key, value in rate_params.items()}

# Run simulation
results = run_simulation(initial_state, rates, number_of_steps, dehydration_rate)
ratios = calculate_ratios(results)
plot_results(results, ratios, real_DL)

In [ ]:
#@title Run simulation
results = run_simulation(initial_state, rates, time_steps)
ratios = calculate_ratios(results)
plot_results(results, ratios, real_DL)